In [2]:
import pandas as pd
import requests
import datetime
from geopy.geocoders import OpenMapQuest

In [3]:
sites_location=pd.read_csv('../data/sites_location.csv', delimiter =';')

In [4]:
sites_location.head()

,site_id,City,UTC_offset
0,0,Jacksonville,-4
1,1,London,0
2,2,Phoenix,-7
3,3,Philadelphia,-4
4,4,San Francisco,-7


In [5]:
from geopy.geocoders import OpenMapQuest
def location(my_address):
    geolocator = OpenMapQuest(api_key='LfGsouoBmQGZZAGpkFZzXGyzo27zOFK2')
    location = geolocator.geocode(my_address, timeout=10)
    return (location.latitude, location.longitude)
sites_location['lat_lon']=sites_location['City'].apply(location)

In [6]:
from timezonefinder import TimezoneFinder
def get_timezone(addr):
    tf = TimezoneFinder()
    return tf.timezone_at(lng=addr[1], lat=addr[0])
sites_location['timezone']=sites_location['lat_lon'].apply(get_timezone)


sites_location.to_csv('../data/sites_location_timezone.csv', index=False)

In [7]:
weather = pd.read_csv('../data/weather_test.csv')
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [49]:
df = pd.merge(weather, sites_location[['site_id', 'UTC_offset', 'timezone']], on = 'site_id', how = 'left')
df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,UTC_offset,timezone
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,-4,America/New_York
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1,-4,America/New_York
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1,-4,America/New_York
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1,-4,America/New_York
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6,-4,America/New_York


In [50]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [51]:
df['timestamp_adjusted'] = df.groupby('timezone')['timestamp'].transform(lambda x: x.dt.tz_localize('UTC').dt.tz_convert(x.name).dt.tz_localize(None))
df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,UTC_offset,timezone,timestamp_adjusted
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,-4,America/New_York,2016-12-31 19:00:00
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1,-4,America/New_York,2016-12-31 20:00:00
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1,-4,America/New_York,2016-12-31 21:00:00
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1,-4,America/New_York,2016-12-31 22:00:00
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6,-4,America/New_York,2016-12-31 23:00:00


In [52]:
weather['timestamp'] = df['timestamp_adjusted']
weather.drop_duplicates(subset=['site_id', 'timestamp'], inplace=True) 
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-12-31 19:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2016-12-31 20:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2016-12-31 21:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2016-12-31 22:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2016-12-31 23:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [53]:
weather[weather.duplicated(subset=['site_id', 'timestamp'])]

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed


In [54]:
weather.to_csv('../data/weather_test_tz_fix.csv', index=False)

In [55]:
weather.shape

(277213, 9)